In [371]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, quantile_transform
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

In [372]:
df = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Predictive Modelling | MBAN 5110 U /Hackathon/Updated_XYZ_Sportswear_Orders_Dataset.csv')

In [373]:
df.head()

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order Type,Customer Order Code,Value,Currency,Items,Route,Value per Item
0,01.01.2009,18.02.2009,DE,PK003,Economy Parka,VO,COD00001,269.87,EUR,8,East,33.733750
1,01.01.2009,10.02.2009,FR,PK001,Advanced Parka,VO,COD00002,170.34,EUR,2,West,85.170000
2,01.01.2009,17.01.2009,ES,PK002,Premium Parka,VO,COD00003,61.09,EUR,7,West,8.727143
3,03.01.2009,28.01.2009,IT,PK003,Economy Parka,VO,COD00004,251.18,EUR,6,North,41.863333
4,03.01.2009,18.01.2009,DE,PK004,Basic Parka,VO,COD00005,153.33,EUR,5,West,30.666000


In [374]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d.%m.%Y')  # Adjust the format as per your data
df['Requested Delivery Date'] = pd.to_datetime(df['Requested Delivery Date'], format='%d.%m.%Y')  # Adjust the format

# Extract month and year from 'Order Date'
df['Order Month'] = df['Order Date'].dt.month
df['Order Year'] = df['Order Date'].dt.year

# Extract month and year from 'Requested Delivery Date'
df['Delivery Month'] = df['Requested Delivery Date'].dt.month
df['Delivery Year'] = df['Requested Delivery Date'].dt.year

In [375]:
df.head()

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order Type,Customer Order Code,Value,Currency,Items,Route,Value per Item,Order Month,Order Year,Delivery Month,Delivery Year
0,2009-01-01,2009-02-18,DE,PK003,Economy Parka,VO,COD00001,269.87,EUR,8,East,33.733750,1,2009,2,2009
1,2009-01-01,2009-02-10,FR,PK001,Advanced Parka,VO,COD00002,170.34,EUR,2,West,85.170000,1,2009,2,2009
2,2009-01-01,2009-01-17,ES,PK002,Premium Parka,VO,COD00003,61.09,EUR,7,West,8.727143,1,2009,1,2009
3,2009-01-03,2009-01-28,IT,PK003,Economy Parka,VO,COD00004,251.18,EUR,6,North,41.863333,1,2009,1,2009
4,2009-01-03,2009-01-18,DE,PK004,Basic Parka,VO,COD00005,153.33,EUR,5,West,30.666000,1,2009,1,2009


In [376]:
monthly_orders = df.groupby(['Order Year', 'Order Month']).agg({'Value': 'sum'}).reset_index()

monthly_orders = monthly_orders.sort_values(by=['Order Year', 'Order Month'])


In [377]:
# Function to map month to season
def month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

# Apply the function to the dataframe
df['Season'] = df['Order Month'].apply(month_to_season)


In [378]:
# Create a combined 'Year-Season' column
df['Year-Season'] = df['Order Year'].astype(str) + '-' + df['Season']

# Aggregate order values by 'Year-Season'
year_season_orders = df.groupby('Year-Season').agg({'Value': 'sum'}).reset_index()

# Sort the data
year_season_orders['Year-Season'] = pd.Categorical(year_season_orders['Year-Season'], categories=sorted(df['Year-Season'].unique()), ordered=True)
year_season_orders = year_season_orders.sort_values('Year-Season')



In [379]:
df.head()

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order Type,Customer Order Code,Value,Currency,Items,Route,Value per Item,Order Month,Order Year,Delivery Month,Delivery Year,Season,Year-Season
0,2009-01-01,2009-02-18,DE,PK003,Economy Parka,VO,COD00001,269.87,EUR,8,East,33.733750,1,2009,2,2009,Winter,2009-Winter
1,2009-01-01,2009-02-10,FR,PK001,Advanced Parka,VO,COD00002,170.34,EUR,2,West,85.170000,1,2009,2,2009,Winter,2009-Winter
2,2009-01-01,2009-01-17,ES,PK002,Premium Parka,VO,COD00003,61.09,EUR,7,West,8.727143,1,2009,1,2009,Winter,2009-Winter
3,2009-01-03,2009-01-28,IT,PK003,Economy Parka,VO,COD00004,251.18,EUR,6,North,41.863333,1,2009,1,2009,Winter,2009-Winter
4,2009-01-03,2009-01-18,DE,PK004,Basic Parka,VO,COD00005,153.33,EUR,5,West,30.666000,1,2009,1,2009,Winter,2009-Winter


In [380]:
df.columns

Index(['Order Date', 'Requested Delivery Date', 'Customer Country Code',
       'Product Code', 'Description', 'Order Type', 'Customer Order Code',
       'Value', 'Currency', 'Items', 'Route', 'Value per Item', 'Order Month',
       'Order Year', 'Delivery Month', 'Delivery Year', 'Season',
       'Year-Season'],
      dtype='object')

In [381]:
df['Lead Time'] = (df['Requested Delivery Date'] - df['Order Date']).dt.days

print(df[['Order Date', 'Requested Delivery Date', 'Lead Time']].head())


  Order Date Requested Delivery Date  Lead Time
0 2009-01-01              2009-02-18         48
1 2009-01-01              2009-02-10         40
2 2009-01-01              2009-01-17         16
3 2009-01-03              2009-01-28         25
4 2009-01-03              2009-01-18         15


In [382]:
# Calculate the difference between 'Requested Delivery Date' and 'Order Date'
df['Lead Time'] = (df['Requested Delivery Date'] - df['Order Date']).dt.days

# Display the first few rows to verify the new column
print(df[['Order Date', 'Requested Delivery Date', 'Lead Time']].head())


  Order Date Requested Delivery Date  Lead Time
0 2009-01-01              2009-02-18         48
1 2009-01-01              2009-02-10         40
2 2009-01-01              2009-01-17         16
3 2009-01-03              2009-01-28         25
4 2009-01-03              2009-01-18         15


In [383]:
df['Value/Item'] = df['Value']/df['Items']

In [384]:
df['Customer_ID'] = df['Customer Country Code'] + '-' + df['Route']

In [385]:
df.head()

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order Type,Customer Order Code,Value,Currency,Items,...,Value per Item,Order Month,Order Year,Delivery Month,Delivery Year,Season,Year-Season,Lead Time,Value/Item,Customer_ID
0,2009-01-01,2009-02-18,DE,PK003,Economy Parka,VO,COD00001,269.87,EUR,8,...,33.733750,1,2009,2,2009,Winter,2009-Winter,48,33.733750,DE-East
1,2009-01-01,2009-02-10,FR,PK001,Advanced Parka,VO,COD00002,170.34,EUR,2,...,85.170000,1,2009,2,2009,Winter,2009-Winter,40,85.170000,FR-West
2,2009-01-01,2009-01-17,ES,PK002,Premium Parka,VO,COD00003,61.09,EUR,7,...,8.727143,1,2009,1,2009,Winter,2009-Winter,16,8.727143,ES-West
3,2009-01-03,2009-01-28,IT,PK003,Economy Parka,VO,COD00004,251.18,EUR,6,...,41.863333,1,2009,1,2009,Winter,2009-Winter,25,41.863333,IT-North
4,2009-01-03,2009-01-18,DE,PK004,Basic Parka,VO,COD00005,153.33,EUR,5,...,30.666000,1,2009,1,2009,Winter,2009-Winter,15,30.666000,DE-West


In [386]:
df.columns

Index(['Order Date', 'Requested Delivery Date', 'Customer Country Code',
       'Product Code', 'Description', 'Order Type', 'Customer Order Code',
       'Value', 'Currency', 'Items', 'Route', 'Value per Item', 'Order Month',
       'Order Year', 'Delivery Month', 'Delivery Year', 'Season',
       'Year-Season', 'Lead Time', 'Value/Item', 'Customer_ID'],
      dtype='object')

In [387]:
df.head()

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order Type,Customer Order Code,Value,Currency,Items,...,Value per Item,Order Month,Order Year,Delivery Month,Delivery Year,Season,Year-Season,Lead Time,Value/Item,Customer_ID
0,2009-01-01,2009-02-18,DE,PK003,Economy Parka,VO,COD00001,269.87,EUR,8,...,33.733750,1,2009,2,2009,Winter,2009-Winter,48,33.733750,DE-East
1,2009-01-01,2009-02-10,FR,PK001,Advanced Parka,VO,COD00002,170.34,EUR,2,...,85.170000,1,2009,2,2009,Winter,2009-Winter,40,85.170000,FR-West
2,2009-01-01,2009-01-17,ES,PK002,Premium Parka,VO,COD00003,61.09,EUR,7,...,8.727143,1,2009,1,2009,Winter,2009-Winter,16,8.727143,ES-West
3,2009-01-03,2009-01-28,IT,PK003,Economy Parka,VO,COD00004,251.18,EUR,6,...,41.863333,1,2009,1,2009,Winter,2009-Winter,25,41.863333,IT-North
4,2009-01-03,2009-01-18,DE,PK004,Basic Parka,VO,COD00005,153.33,EUR,5,...,30.666000,1,2009,1,2009,Winter,2009-Winter,15,30.666000,DE-West


In [388]:
# Mapping of seasons to numerical representations
season_to_number = {'Winter': '1', 'Spring': '2', 'Summer': '3', 'Fall': '4'}

# Create 'Year-SeasonNumber' column
df['Year-SeasonNumber'] = df['Order Year'].astype(str) + '-' + df['Season'].map(season_to_number)

# Verify the changes
df[['Year-Season', 'Year-SeasonNumber']].head()


,Year-Season,Year-SeasonNumber
0,2009-Winter,2009-1
1,2009-Winter,2009-1
2,2009-Winter,2009-1
3,2009-Winter,2009-1
4,2009-Winter,2009-1


In [389]:
df.columns

Index(['Order Date', 'Requested Delivery Date', 'Customer Country Code',
       'Product Code', 'Description', 'Order Type', 'Customer Order Code',
       'Value', 'Currency', 'Items', 'Route', 'Value per Item', 'Order Month',
       'Order Year', 'Delivery Month', 'Delivery Year', 'Season',
       'Year-Season', 'Lead Time', 'Value/Item', 'Customer_ID',
       'Year-SeasonNumber'],
      dtype='object')

In [390]:
df.head()

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order Type,Customer Order Code,Value,Currency,Items,...,Order Month,Order Year,Delivery Month,Delivery Year,Season,Year-Season,Lead Time,Value/Item,Customer_ID,Year-SeasonNumber
0,2009-01-01,2009-02-18,DE,PK003,Economy Parka,VO,COD00001,269.87,EUR,8,...,1,2009,2,2009,Winter,2009-Winter,48,33.733750,DE-East,2009-1
1,2009-01-01,2009-02-10,FR,PK001,Advanced Parka,VO,COD00002,170.34,EUR,2,...,1,2009,2,2009,Winter,2009-Winter,40,85.170000,FR-West,2009-1
2,2009-01-01,2009-01-17,ES,PK002,Premium Parka,VO,COD00003,61.09,EUR,7,...,1,2009,1,2009,Winter,2009-Winter,16,8.727143,ES-West,2009-1
3,2009-01-03,2009-01-28,IT,PK003,Economy Parka,VO,COD00004,251.18,EUR,6,...,1,2009,1,2009,Winter,2009-Winter,25,41.863333,IT-North,2009-1
4,2009-01-03,2009-01-18,DE,PK004,Basic Parka,VO,COD00005,153.33,EUR,5,...,1,2009,1,2009,Winter,2009-Winter,15,30.666000,DE-West,2009-1


In [391]:
df['Season'].unique()

array(['Winter', 'Spring', 'Summer', 'Fall'], dtype=object)

In [392]:
df['Season_Num'] = df['Season'].map({'Winter': 1, 'Spring': 2, 'Summer': 3, 'Fall': 4})

In [393]:
df['Season_Num'].unique()

array([1, 2, 3, 4])

In [394]:
df.head()

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order Type,Customer Order Code,Value,Currency,Items,...,Order Year,Delivery Month,Delivery Year,Season,Year-Season,Lead Time,Value/Item,Customer_ID,Year-SeasonNumber,Season_Num
0,2009-01-01,2009-02-18,DE,PK003,Economy Parka,VO,COD00001,269.87,EUR,8,...,2009,2,2009,Winter,2009-Winter,48,33.733750,DE-East,2009-1,1
1,2009-01-01,2009-02-10,FR,PK001,Advanced Parka,VO,COD00002,170.34,EUR,2,...,2009,2,2009,Winter,2009-Winter,40,85.170000,FR-West,2009-1,1
2,2009-01-01,2009-01-17,ES,PK002,Premium Parka,VO,COD00003,61.09,EUR,7,...,2009,1,2009,Winter,2009-Winter,16,8.727143,ES-West,2009-1,1
3,2009-01-03,2009-01-28,IT,PK003,Economy Parka,VO,COD00004,251.18,EUR,6,...,2009,1,2009,Winter,2009-Winter,25,41.863333,IT-North,2009-1,1
4,2009-01-03,2009-01-18,DE,PK004,Basic Parka,VO,COD00005,153.33,EUR,5,...,2009,1,2009,Winter,2009-Winter,15,30.666000,DE-West,2009-1,1


In [395]:
df = df[['Customer Order Code','Order Date', 'Order Month', 'Order Year', 'Requested Delivery Date', 'Delivery Month', 'Delivery Year','Lead Time', 'Year-Season', 'Season', 'Season_Num', 
         'Customer Country Code',  'Route',  'Customer_ID',  'Order Type',  
         'Product Code',  'Description',  'Items', 'Value', 'Value/Item', 'Currency'   
        ]]

In [396]:
df.head()

,Customer Order Code,Order Date,Order Month,Order Year,Requested Delivery Date,Delivery Month,Delivery Year,Lead Time,Year-Season,Season,...,Customer Country Code,Route,Customer_ID,Order Type,Product Code,Description,Items,Value,Value/Item,Currency
0,COD00001,2009-01-01,1,2009,2009-02-18,2,2009,48,2009-Winter,Winter,...,DE,East,DE-East,VO,PK003,Economy Parka,8,269.87,33.733750,EUR
1,COD00002,2009-01-01,1,2009,2009-02-10,2,2009,40,2009-Winter,Winter,...,FR,West,FR-West,VO,PK001,Advanced Parka,2,170.34,85.170000,EUR
2,COD00003,2009-01-01,1,2009,2009-01-17,1,2009,16,2009-Winter,Winter,...,ES,West,ES-West,VO,PK002,Premium Parka,7,61.09,8.727143,EUR
3,COD00004,2009-01-03,1,2009,2009-01-28,1,2009,25,2009-Winter,Winter,...,IT,North,IT-North,VO,PK003,Economy Parka,6,251.18,41.863333,EUR
4,COD00005,2009-01-03,1,2009,2009-01-18,1,2009,15,2009-Winter,Winter,...,DE,West,DE-West,VO,PK004,Basic Parka,5,153.33,30.666000,EUR


In [397]:
df['Product Code'].unique()

array(['PK003', 'PK001', 'PK002', 'PK004'], dtype=object)

In [398]:
df.columns

Index(['Customer Order Code', 'Order Date', 'Order Month', 'Order Year',
       'Requested Delivery Date', 'Delivery Month', 'Delivery Year',
       'Lead Time', 'Year-Season', 'Season', 'Season_Num',
       'Customer Country Code', 'Route', 'Customer_ID', 'Order Type',
       'Product Code', 'Description', 'Items', 'Value', 'Value/Item',
       'Currency'],
      dtype='object')

In [399]:
product_dummies = pd.get_dummies(df['Product Code'])
df = pd.concat([df, product_dummies], axis=1)

In [400]:
df.head()

,Customer Order Code,Order Date,Order Month,Order Year,Requested Delivery Date,Delivery Month,Delivery Year,Lead Time,Year-Season,Season,...,Product Code,Description,Items,Value,Value/Item,Currency,PK001,PK002,PK003,PK004
0,COD00001,2009-01-01,1,2009,2009-02-18,2,2009,48,2009-Winter,Winter,...,PK003,Economy Parka,8,269.87,33.733750,EUR,0,0,1,0
1,COD00002,2009-01-01,1,2009,2009-02-10,2,2009,40,2009-Winter,Winter,...,PK001,Advanced Parka,2,170.34,85.170000,EUR,1,0,0,0
2,COD00003,2009-01-01,1,2009,2009-01-17,1,2009,16,2009-Winter,Winter,...,PK002,Premium Parka,7,61.09,8.727143,EUR,0,1,0,0
3,COD00004,2009-01-03,1,2009,2009-01-28,1,2009,25,2009-Winter,Winter,...,PK003,Economy Parka,6,251.18,41.863333,EUR,0,0,1,0
4,COD00005,2009-01-03,1,2009,2009-01-18,1,2009,15,2009-Winter,Winter,...,PK004,Basic Parka,5,153.33,30.666000,EUR,0,0,0,1


In [401]:
df.columns

Index(['Customer Order Code', 'Order Date', 'Order Month', 'Order Year',
       'Requested Delivery Date', 'Delivery Month', 'Delivery Year',
       'Lead Time', 'Year-Season', 'Season', 'Season_Num',
       'Customer Country Code', 'Route', 'Customer_ID', 'Order Type',
       'Product Code', 'Description', 'Items', 'Value', 'Value/Item',
       'Currency', 'PK001', 'PK002', 'PK003', 'PK004'],
      dtype='object')

In [407]:
df.columns

Index(['Customer Order Code', 'Order Date', 'Order Month', 'Order Year',
       'Requested Delivery Date', 'Delivery Month', 'Delivery Year',
       'Lead Time', 'Year-Season', 'Season', 'Season_Num',
       'Customer Country Code', 'Route', 'Customer_ID', 'Order Type',
       'Product Code', 'Description', 'Items', 'Value', 'Value/Item',
       'Currency', 'PK001', 'PK002', 'PK003', 'PK004'],
      dtype='object')

In [414]:
categorical_features = ['Customer Order Code', 'Season', 'Order Month', 'Order Year']  # Update this as necessary
numerical_features = ['Lead Time']  # Update this as necessary

In [415]:
cat_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
num_transformer = Pipeline(steps=[('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')), ('scaler', StandardScaler())])


preprocessor = ColumnTransformer(transformers=[
    ('cat', cat_transformer, categorical_features),
    ('num', num_transformer, numerical_features)])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression(max_iter=1000, multi_class='multinomial'))])

y = df['PK003']

# Make sure X only contains the features you defined, without the target variables or any columns you've dropped
X = df[categorical_features + numerical_features]

# Now you can fit the pipeline
pipeline.fit(X, y)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Customer Order Code',
                                                   'Season', 'Order Month',
                                                   'Order Year']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Lead Time'])])),
                ('classifier',
                 LogisticRegression(max_iter=1000, multi_class='multinomial'))])

In [416]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Predict the probabilities for the test data
probabilities = pipeline.predict_proba(X_test)

# The probabilities are in a 2D array, with one column for each class. The second column is the probability of the positive class.
choice_probability = probabilities[:, 1]

# Predict the class labels for the test set
y_pred = pipeline.predict(X_test)

# Calculate the accuracy for the test set
accuracy = accuracy_score(y_test, y_pred)


# Print the accuracy
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.77


In [418]:
from sklearn.preprocessing import LabelEncoder

# Convert 'Product Code' into numerical categories
le = LabelEncoder()
y = le.fit_transform(df['Product Code'])

# Adjust the logistic regression model for multinomial classification
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression(max_iter=1000, multi_class='multinomial'))])

# Fit the pipeline to the entire dataset
pipeline.fit(X, y)

# Predict the probabilities for each class
probabilities = pipeline.predict_proba(X)

# Create a new DataFrame with the predicted probabilities and ensure the columns don't overlap with existing columns
prob_df = pd.DataFrame(probabilities, columns=[f'prob_{code}' for code in le.classes_])

# Join this DataFrame with the original DataFrame
# Since the original DataFrame contains indices, reset_index to align both DataFrames by index
df_with_probs = df.reset_index().join(prob_df)

# Now, if you want to calculate the mean probability for each product code per customer,
# you can group by 'Customer_ID' and calculate the mean for these new probability columns
choice_probabilities = df_with_probs.groupby('Customer_ID')[[f'prob_{code}' for code in le.classes_]].mean()


In [420]:
choice_probabilities.head()

,prob_PK001,prob_PK002,prob_PK003,prob_PK004
Customer_ID,,,,
DE-East,0.245628,0.242531,0.245070,0.266771
DE-North,0.263369,0.238471,0.253303,0.244857
DE-South,0.265906,0.244035,0.247593,0.242466
DE-West,0.269851,0.237828,0.238882,0.253438
ES-East,0.263569,0.230124,0.244566,0.261741


In [424]:
le = LabelEncoder()
y = le.fit_transform(df['Product Code'])
X = df.drop(columns=['PK001', 'PK002', 'PK003', 'PK004', 'Product Code'])

In [425]:
pipeline.fit(X, y)

probabilities = pipeline.predict_proba(X)

prob_df = pd.DataFrame(probabilities, columns=[f'prob_{code}' for code in le.classes_])

df_with_probs = df.reset_index(drop=True).join(prob_df)

choice_probabilities = df_with_probs.groupby(['Customer_ID', 'Season_Num'])[[f'prob_{code}' for code in le.classes_]].mean()
choice_probabilities = choice_probabilities.reset_index()


In [430]:
choice_probabilities.head(20)

,Customer_ID,Season_Num,prob_PK001,prob_PK002,prob_PK003,prob_PK004
0,DE-East,1,0.220485,0.232545,0.273058,0.273912
1,DE-East,2,0.268481,0.246832,0.248293,0.236395
2,DE-East,3,0.240284,0.260221,0.210747,0.288748
3,DE-East,4,0.245769,0.225853,0.246002,0.282375
4,DE-North,1,0.274766,0.203040,0.263408,0.258787
5,DE-North,2,0.280312,0.249420,0.238452,0.231816
6,DE-North,3,0.217502,0.293185,0.249759,0.239554
7,DE-North,4,0.274236,0.206382,0.267315,0.252067
8,DE-South,1,0.261886,0.232721,0.266444,0.238948
9,DE-South,2,0.303926,0.273905,0.192875,0.229294


In [427]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Predict the probabilities on the test set
test_probabilities = pipeline.predict_proba(X_test)

# Calculate log loss on the test set
# Log loss is a measure of accuracy for categorical classifiers, lower is better
test_log_loss = log_loss(y_test, test_probabilities)
print(f'Log Loss on the Test Set: {test_log_loss:.4f}')

# If you also want to calculate accuracy, though it is less informative for probabilistic predictions
# Predict the class labels for the test set
y_pred = pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy on the Test Set: {test_accuracy:.4f}')


Log Loss on the Test Set: 1.3967
Accuracy on the Test Set: 0.2742


In [431]:
# Assuming 'Items' is the column with the quantity of each product ordered
# We want to create a new row for each item ordered
df_expanded = df.loc[df.index.repeat(df['Items'])].copy()
df_expanded['Items'] = 1  # Set 'Items' to 1 since each row is now a single item

# You would then create binary outcomes for each product and proceed with model fitting as before


In [435]:
df_expanded.head(20)

,Customer Order Code,Order Date,Order Month,Order Year,Requested Delivery Date,Delivery Month,Delivery Year,Lead Time,Year-Season,Season,...,Product Code,Description,Items,Value,Value/Item,Currency,PK001,PK002,PK003,PK004
0,COD00001,2009-01-01,1,2009,2009-02-18,2,2009,48,2009-Winter,Winter,...,PK003,Economy Parka,1,269.87,33.733750,EUR,0,0,1,0
0,COD00001,2009-01-01,1,2009,2009-02-18,2,2009,48,2009-Winter,Winter,...,PK003,Economy Parka,1,269.87,33.733750,EUR,0,0,1,0
0,COD00001,2009-01-01,1,2009,2009-02-18,2,2009,48,2009-Winter,Winter,...,PK003,Economy Parka,1,269.87,33.733750,EUR,0,0,1,0
0,COD00001,2009-01-01,1,2009,2009-02-18,2,2009,48,2009-Winter,Winter,...,PK003,Economy Parka,1,269.87,33.733750,EUR,0,0,1,0
0,COD00001,2009-01-01,1,2009,2009-02-18,2,2009,48,2009-Winter,Winter,...,PK003,Economy Parka,1,269.87,33.733750,EUR,0,0,1,0
0,COD00001,2009-01-01,1,2009,2009-02-18,2,2009,48,2009-Winter,Winter,...,PK003,Economy Parka,1,269.87,33.733750,EUR,0,0,1,0
0,COD00001,2009-01-01,1,2009,2009-02-18,2,2009,48,2009-Winter,Winter,...,PK003,Economy Parka,1,269.87,33.733750,EUR,0,0,1,0
0,COD00001,2009-01-01,1,2009,2009-02-18,2,2009,48,2009-Winter,Winter,...,PK003,Economy Parka,1,269.87,33.733750,EUR,0,0,1,0
1,COD00002,2009-01-01,1,2009,2009-02-10,2,2009,40,2009-Winter,Winter,...,PK001,Advanced Parka,1,170.34,85.170000,EUR,1,0,0,0
1,COD00002,2009-01-01,1,2009,2009-02-10,2,2009,40,2009-Winter,Winter,...,PK001,Advanced Parka,1,170.34,85.170000,EUR,1,0,0,0


In [436]:
df.columns

Index(['Customer Order Code', 'Order Date', 'Order Month', 'Order Year',
       'Requested Delivery Date', 'Delivery Month', 'Delivery Year',
       'Lead Time', 'Year-Season', 'Season', 'Season_Num',
       'Customer Country Code', 'Route', 'Customer_ID', 'Order Type',
       'Product Code', 'Description', 'Items', 'Value', 'Value/Item',
       'Currency', 'PK001', 'PK002', 'PK003', 'PK004'],
      dtype='object')

In [440]:
# Selecting features and target for the model
features = ['Order Month', 'Order Year', 'Delivery Month', 'Delivery Year', 'Lead Time', 'Season_Num','Customer_ID']
target = 'Product Code'  # This should be the categorical variable with actual product codes

# Preprocessing for categorical variables
categorical_features = ['Order Month', 'Season_Num', 'Customer_ID']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Preprocessing for numerical variables
numerical_features = ['Order Year', 'Delivery Month', 'Delivery Year', 'Lead Time']
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# Create the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

# Create the pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, multi_class='multinomial'))])

# Prepare the features matrix and target vector
X = df_expanded[features]
y = df_expanded[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Fit the model
model.fit(X_train, y_train)

# Predict probabiliti
probabilities = model.predict_proba(X_test)

In [441]:
# Add probabilities back to the test set
prob_cols = [f'prob_{i}' for i in range(probabilities.shape[1])]
df_probs = pd.DataFrame(probabilities, columns=prob_cols, index=X_test.index)
df_test_with_probs = X_test.join(df_probs)

# Assuming Customer_ID and Season_Num are in the test set, group by them and calculate mean probabilities
choice_probabilities = df_test_with_probs.groupby(['Customer_ID', 'Season_Num'])[prob_cols].mean()

# Reset index if needed
choice_probabilities.reset_index(inplace=True)


In [443]:
choice_probabilities.head(20)

,Customer_ID,Season_Num,prob_0,prob_1,prob_2,prob_3
0,DE-East,1,0.215303,0.240317,0.284547,0.259834
1,DE-East,2,0.217154,0.251682,0.254361,0.276803
2,DE-East,3,0.176232,0.246808,0.240415,0.336545
3,DE-East,4,0.188397,0.189625,0.287319,0.334659
4,DE-North,1,0.307878,0.189741,0.351456,0.150925
5,DE-North,2,0.292784,0.212225,0.328383,0.166609
6,DE-North,3,0.254901,0.220901,0.323564,0.200634
7,DE-North,4,0.279740,0.154221,0.366629,0.199410
8,DE-South,1,0.313419,0.215833,0.270157,0.200592
9,DE-South,2,0.277225,0.240059,0.268809,0.213907


In [447]:
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

# True labels of the test set
# Make sure y_test is encoded with the same LabelEncoder instance used for y_train
y_true = le.transform(y_test)

# Predicted probabilities
y_pred_probs = model.predict_proba(X_test)

# Calculate and print log loss
logloss = log_loss(y_true, y_pred_probs)
print(f'Log Loss: {logloss:.4f}')

# Predict the class labels for the test set
y_pred = model.predict(X_test)

# Make sure y_pred is also encoded if necessary
# If y_pred is not encoded, you can use the inverse_transform method to convert it back to the original label
#y_pred = le.inverse_transform(y_pred)

# Calculate and print accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Print the confusion matrix
cm = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:\n', cm)

# Plot the calibration curve
# Ensure you are using the correct column from y_pred_probs, this should match the class encoding
# If using all classes, you'll need to calibrate for each one or use a multiclass calibration method
prob_true, prob_pred = calibration_curve(y_true, y_pred_probs[:, 1], n_bins=10)  # Adjust index 1 as needed
plt.plot(prob_pred, prob_true, marker='o', linewidth=1, label='calibration plot')
plt.plot([0, 1], [0, 1], linestyle='--', label='perfectly calibrated')
plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')
plt.legend()
plt.show()



Log Loss: 1.3623
Accuracy: 0.0000


ValueError: Mix of label input types (string and number)

In [448]:
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

# True labels of the test set should be encoded with the same LabelEncoder instance used for y_train
y_true = le.transform(y_test)

# Predicted probabilities
y_pred_probs = model.predict_proba(X_test)

# Calculate and print log loss
logloss = log_loss(y_true, y_pred_probs)
print(f'Log Loss: {logloss:.4f}')

# Predict the class labels for the test set
y_pred = model.predict(X_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Print the confusion matrix
cm = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:\n', cm)

# Calibration curve requires the true labels and predicted probabilities
# Here you need to make sure you are using the correct probabilities for the calibration curve
# If you're comparing all classes, you should do this once for each class or use a multiclass calibration method
for i, class_label in enumerate(le.classes_):
    # Only select the probabilities for the class i
    prob_true, prob_pred = calibration_curve(y_true == i, y_pred_probs[:, i], n_bins=10)
    
    # Plot the calibration curve for class i
    plt.figure(i)
    plt.plot(prob_pred, prob_true, marker='o', linewidth=1, label=f'calibration plot (class {class_label})')
    plt.plot([0, 1], [0, 1], linestyle='--', label='perfectly calibrated')
    plt.xlabel('Mean predicted probability')
    plt.ylabel('Fraction of positives')
    plt.title(f'Calibration curve for {class_label}')
    plt.legend()
    plt.show()


Log Loss: 1.3623
Accuracy: 0.0000


ValueError: Mix of label input types (string and number)